# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse    Shipments             Customer  \
0  Jun 15 2022  2:25PM  239679        19  ADV80003354  AdvaGen Pharma, Ltd   
1  Jun 15 2022  2:25PM  239679        19  ADV80003355  AdvaGen Pharma, Ltd   
2  Jun 15 2022  2:25PM  239679        19  ADV80003356  AdvaGen Pharma, Ltd   
3  Jun 15 2022  2:25PM  239679        19  ADV80003357  AdvaGen Pharma, Ltd   
4  Jun 15 2022  2:25PM  239679        19  ADV80003358  AdvaGen Pharma, Ltd   
5  Jun 15 2022  2:25PM  239679        19  ADV80003359  AdvaGen Pharma, Ltd   
6  Jun 15 2022  2:25PM  239679        19  ADV80003360  AdvaGen Pharma, Ltd   
7  Jun 15 2022  2:25PM  239679        19  ADV80003361  AdvaGen Pharma, Ltd   
8  Jun 15 2022  2:25PM  239679        19  ADV80003362  AdvaGen Pharma, Ltd   
9  Jun 15 2022  2:25PM  239679        19  ADV80003363  AdvaGen Pharma, Ltd   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
50  239673       Released          1
51  239674       Released          1
52  239675       Released          1
53  239677       Released          5
54  239679       Released         14

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
239673                NaN        NaN       1.0
239674                NaN        NaN       1.0
239675                NaN        NaN       1.0
239677                NaN        NaN       5.0
239679                NaN        NaN      14.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
239587                0.0        0.0       1.0
239590                0.0        0.0       1.0
239593                0.0        1.0       0.0
239595                0.0        1.0       0.0
239608                6.0        5.0       4.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
239587                  0          0         1
239590                  0          0         1
239593                  0          1         0
239595                  0          1         0
239608                  6          5         4

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               239587          0          0         1
1               239590          0          0         1
2               239593          0          1         0
3               239595          0          1         0
4               239608          6          5         4

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               239587                            1
1               239590                            1
2               239593                   1         
3               239595                   1         
4               239608         6         5        4

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse               Customer
0    Jun 15 2022  2:25PM  239679        19    AdvaGen Pharma, Ltd
14   Jun 15 2022  2:18PM  239677        10       Methapharm, Inc.
19   Jun 15 2022  2:17PM  239675        10      ISDIN Corporation
20   Jun 15 2022  2:11PM  239674        20         Exact-Rx, Inc.
21   Jun 15 2022  2:09PM  239672        20      Alumier Labs Inc.
50   Jun 15 2022  2:09PM  239673        19  ACG North America LLC
51   Jun 15 2022  2:05PM  239653        10      ISDIN Corporation
103  Jun 15 2022  2:05PM  239647        10      ISDIN Corporation
138  Jun 15 2022  2:05PM  239639        10      ISDIN Corporation
181  Jun 15 2022  2:03PM  239671        10       Methapharm, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse               Customer Completed  \
0  Jun 15 2022  2:25PM  239679        19    AdvaGen Pharma, Ltd             
1  Jun 15 2022  2:18PM  239677        10       Methapharm, Inc.             
2  Jun 15 2022  2:17PM  239675        10      ISDIN Corporation             
3  Jun 15 2022  2:11PM  239674        20         Exact-Rx, Inc.             
4  Jun 15 2022  2:09PM  239672        20      Alumier Labs Inc.             
5  Jun 15 2022  2:09PM  239673        19  ACG North America LLC             
6  Jun 15 2022  2:05PM  239653        10      ISDIN Corporation             
7  Jun 15 2022  2:05PM  239647        10      ISDIN Corporation             
8  Jun 15 2022  2:05PM  239639        10      ISDIN Corporation             
9  Jun 15 2022  2:03PM  239671        10       Methapharm, Inc.             

  Executing Released  
0                 14  
1                  5  
2                  1  
3                  1  
4                 29  
5                  1  
6                 52  
7                 35  
8                 43  
9                  7

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse               Customer Released  \
0  Jun 15 2022  2:25PM  239679        19    AdvaGen Pharma, Ltd       14   
1  Jun 15 2022  2:18PM  239677        10       Methapharm, Inc.        5   
2  Jun 15 2022  2:17PM  239675        10      ISDIN Corporation        1   
3  Jun 15 2022  2:11PM  239674        20         Exact-Rx, Inc.        1   
4  Jun 15 2022  2:09PM  239672        20      Alumier Labs Inc.       29   
5  Jun 15 2022  2:09PM  239673        19  ACG North America LLC        1   
6  Jun 15 2022  2:05PM  239653        10      ISDIN Corporation       52   
7  Jun 15 2022  2:05PM  239647        10      ISDIN Corporation       35   
8  Jun 15 2022  2:05PM  239639        10      ISDIN Corporation       43   
9  Jun 15 2022  2:03PM  239671        10       Methapharm, Inc.        7   

  Executing Completed  
0                      
1                      
2                      
3                      
4                      
5                      
6                      
7                      
8                      
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse             Customer Released  \
0  Jun 15 2022  2:25PM  239679        19  AdvaGen Pharma, Ltd       14   
1  Jun 15 2022  2:18PM  239677        10     Methapharm, Inc.        5   
2  Jun 15 2022  2:17PM  239675        10    ISDIN Corporation        1   
3  Jun 15 2022  2:11PM  239674        20       Exact-Rx, Inc.        1   
4  Jun 15 2022  2:09PM  239672        20    Alumier Labs Inc.       29   

  Executing Completed  
0                      
1                      
2                      
3                      
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse             Customer  Released  \
0  Jun 15 2022  2:25PM  239679        19  AdvaGen Pharma, Ltd      14.0   
1  Jun 15 2022  2:18PM  239677        10     Methapharm, Inc.       5.0   
2  Jun 15 2022  2:17PM  239675        10    ISDIN Corporation       1.0   
3  Jun 15 2022  2:11PM  239674        20       Exact-Rx, Inc.       1.0   
4  Jun 15 2022  2:09PM  239672        20    Alumier Labs Inc.      29.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse             Customer  Released  \
0  Jun 15 2022  2:25PM  239679        19  AdvaGen Pharma, Ltd      14.0   
1  Jun 15 2022  2:18PM  239677        10     Methapharm, Inc.       5.0   
2  Jun 15 2022  2:17PM  239675        10    ISDIN Corporation       1.0   
3  Jun 15 2022  2:11PM  239674        20       Exact-Rx, Inc.       1.0   
4  Jun 15 2022  2:09PM  239672        20    Alumier Labs Inc.      29.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3355146     204.0       36.0       25.0
12          479244       2.0        1.0        0.0
15          718938       8.0        0.0        0.0
16          239651       3.0        0.0        0.0
18          718858       3.0        0.0        0.0
19         1917091      28.0        9.0        8.0
20         1437970      94.0       32.0        6.0
21          479188       0.0        2.0        0.0
22          718977       3.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3355146     204.0       36.0       25.0
1        12   479244       2.0        1.0        0.0
2        15   718938       8.0        0.0        0.0
3        16   239651       3.0        0.0        0.0
4        18   718858       3.0        0.0        0.0
5        19  1917091      28.0        9.0        8.0
6        20  1437970      94.0       32.0        6.0
7        21   479188       0.0        2.0        0.0
8        22   718977       3.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     204.0       36.0       25.0
1        12       2.0        1.0        0.0
2        15       8.0        0.0        0.0
3        16       3.0        0.0        0.0
4        18       3.0        0.0        0.0
5        19      28.0        9.0        8.0
6        20      94.0       32.0        6.0
7        21       0.0        2.0        0.0
8        22       3.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  204.0
1        12  Released    2.0
2        15  Released    8.0
3        16  Released    3.0
4        18  Released    3.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10   12   15   16   18    19    20   21   22
Status                                                    
Completed   25.0  0.0  0.0  0.0  0.0   8.0   6.0  0.0  0.0
Executing   36.0  1.0  0.0  0.0  0.0   9.0  32.0  2.0  0.0
Released   204.0  2.0  8.0  3.0  3.0  28.0  94.0  0.0  3.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10   12   15   16   18    19    20   21   22
0          Completed   25.0  0.0  0.0  0.0  0.0   8.0   6.0  0.0  0.0
1          Executing   36.0  1.0  0.0  0.0  0.0   9.0  32.0  2.0  0.0
2           Released  204.0  2.0  8.0  3.0  3.0  28.0  94.0  0.0  3.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10   12   15   16   18    19    20   21   22
0  Completed   25.0  0.0  0.0  0.0  0.0   8.0   6.0  0.0  0.0
1  Executing   36.0  1.0  0.0  0.0  0.0   9.0  32.0  2.0  0.0
2   Released  204.0  2.0  8.0  3.0  3.0  28.0  94.0  0.0  3.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()